In [ ]:
import cv2
cap = cv2.VideoCapture("test.mp4")

In [17]:
from ultralytics import YOLO
from PIL import Image


# Load the YOLOv8 model
model = YOLO("yolov8m.pt")
model.cpu()

results = model(Image.open("bus.jpg"))
print(results)



100%|██████████| 49.7M/49.7M [00:01<00:00, 29.5MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [42]:
def startValues():
    #names
    names = results[0].names

    #count elements founded
    count = [0]*len(results[0].names)
    return names,count

def countFrame(count):
    for r in results:
        for cls in r.boxes.cls:
            count[int(cls.item())] += 1

def translateLabelNames(names,count):
    #translate elements
    output = {}
    for i in range(len(names)):
        if count[i]:
            output[names[i]] = count[i]
    #Only list of labels
    return list(output.keys())

names,count = startValues()
countFrame(count)
translateLabelNames(names,count)

['person', 'bus', 'stop sign']

In [33]:
# Export the model to ONNX format
#model.export(format="onnx")  # creates 'yolov8n.onnx'

# Load the exported ONNX model
#model = YOLO("yolov8n.onnx")

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [34]:
results = model(Image.open("bus.jpg"))
#names
names = results[0].names

#count elements founded
count = [0]*len(results[0].names)
for r in results:
    for cls in r.boxes.cls:
        count[int(cls.item())] += 1

#translate elements
output = {}
for i in range(len(names)):
    if count[i]:
        output[names[i]] = count[i]
#Only list of labels
print(list(output.keys()))

Loading yolov8n.onnx for ONNX Runtime inference...



InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: images for the following indices
 index: 2 Got: 640 Expected: 224
 index: 3 Got: 640 Expected: 224
 Please fix either the inputs/outputs or the model.

# To lambda

In [ ]:
import json
import cv2
import boto3
from PIL import Image
from io import BytesIO
from torchvision.transforms import functional as F
from ultralytics import YOLO

s3_client = boto3.client('s3')



def startValues(results):
    #names
    names = results[0].names

    #count elements founded
    count = [0]*len(results[0].names)
    return names,count

def countFrame(results,count):
    for r in results:
        for cls in r.boxes.cls:
            count[int(cls.item())] += 1
    return count

def translateLabelNames(names,count):
    #translate elements
    output = {}
    for i in range(len(names)):
        if count[i]:
            output[names[i]] = count[i]
    #Only list of labels
    return list(output.keys())

def lambda_handler(event, context):
    # Obtener el bucket y key del evento
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    # Descargar el archivo de video desde S3
    s3_response = s3_client.get_object(Bucket=bucket, Key=key)
    video_data = s3_response['Body'].read()

    # Leer el video en memoria
    video_stream = BytesIO(video_data)
    cap = cv2.VideoCapture(video_stream)

    # Cargar el modelo YOLOv8
    model = YOLO("/var/task/yolov8m.pt")
    #model.cpu()

    # Procesar el video cuadro por cuadro
    names,count = startValues()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir el cuadro a formato PIL
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Detectar objetos en el cuadro
        results = model(frame_pil)
        count = countFrame(results,count)
        
    # Liberar el video
    cap.release()

    tags = translateLabelNames(names,count)

    # Guardar los resultados en S3
    

    return {
        'statusCode': 200,
        'body': json.dumps('Detección de objetos completada.\nSe encontrarons las siguientes etiquetas'+','.join(tags))
    }
